In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision
import argparse
import random

# distributed training
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DataParallel
from torch.nn.parallel import DistributedDataParallel as DDP

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from efficientnet_pytorch import EfficientNet

from model import load_optimizer, save_model
from modules import SimCLR, NT_Xent, get_resnet, EarlyStopping
from modules.transformations import TransformsSimCLR
from modules.sync_batchnorm import convert_model
from utils import yaml_config_hook

In [7]:
parser = argparse.ArgumentParser(description="SimCLR")
config = yaml_config_hook("./config/config.yaml")
for k, v in config.items():
    parser.add_argument(f"--{k}", default=v, type=type(v))
args_str = '' 
args, _ = parser.parse_known_args(args=args_str)
args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(args.device)
args.num_gpus = torch.cuda.device_count()
args.world_size = args.gpus * args.nodes
gpu = 0
rank = args.nr * args.gpus + gpu

if args.nodes > 1:
        dist.init_process_group("nccl", rank=rank, world_size=args.world_size)
        torch.cuda.set_device(gpu)

cpu


In [3]:
torch.manual_seed(args.seed)
np.random.seed(args.seed)
#------- added by young ---------
torch.cuda.manual_seed(args.seed)
if args.gpus > 1:
    torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(args.seed)
random.seed(args.seed)

In [4]:
train_dataset = torchvision.datasets.ImageFolder(
    '/home/opticho/source/SimCLR/datasets/dataset2(3)/train/train', 
    transform=TransformsSimCLR(size=(args.image_size, args.image_size)).test_transform)
valid_dataset = torchvision.datasets.ImageFolder(
    '/home/opticho/source/SimCLR/datasets/dataset2(3)/train/valid', 
    transform=TransformsSimCLR(size=(args.image_size, args.image_size)).test_transform)
test_dataset = torchvision.datasets.ImageFolder(
    '/home/opticho/source/SimCLR/datasets/dataset2(3)/test', 
    transform=TransformsSimCLR(size=(args.image_size, args.image_size)).test_transform)

In [1]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.logistic_batch_size,
    shuffle=True,
    drop_last=False,
    num_workers=args.workers,
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=args.logistic_batch_size,
    shuffle=True,
    drop_last=False,
    num_workers=args.workers,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=args.logistic_batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=args.workers,
)

NameError: name 'torch' is not defined

In [6]:
class Model(torch.nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.conv2d = torch.nn.Conv2d(3, 3, 3, padding=1)
        self.net = EfficientNet.from_pretrained('efficientnet-b0', num_classes=3, include_top = False)
        self.seq = torch.nn.Sequential(
            nn.BatchNorm1d(1280),
            nn.Dropout(0.5),
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 3)
        )

    def forward(self, x):
        x = self.conv2d(x)
        x = self.net(x)
        x = x.view(-1, 1280)
        x = self.seq(x)
        
        return x

In [41]:
inp = torch.Tensor(32, 3, 244, 244)

out = nn.Conv2d(3, 3, 3, padding=1)(inp)
print(out.shape)
out = EfficientNet.from_pretrained('efficientnet-b0', num_classes=3, include_top = False)(out)
print(out.shape)
out = out.view(-1, 1280)
out = nn.BatchNorm1d(1280)(out)
print(out.shape)
out = nn.Dropout(0.5)(out)
print(out.shape)
out = nn.Linear(1280, 512)(out)
out = nn.ReLU()(out)
print(out.shape)
out = nn.BatchNorm1d(512)(out)
print(out.shape)
out = nn.Dropout(0.5)(out)
print(out.shape)
out = nn.Linear(512, 3)(out)

torch.Size([32, 3, 244, 244])
Loaded pretrained weights for efficientnet-b0
torch.Size([32, 1280, 1, 1])
torch.Size([32, 1280])
torch.Size([32, 1280])
torch.Size([32, 512])
torch.Size([32, 512])
torch.Size([32, 512])


In [7]:
# initialize ResNet
# model = torchvision.models.resnet18(pretrained=True)

# ct = 0 
# for child in model.children(): 
#     ct += 1 
#     if ct < 6:
#         for param in child.parameters():
#             param.requires_grad = False



model = Model()
model.to(args.device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

Loaded pretrained weights for efficientnet-b0


In [8]:
epochs = 10000
device = args.device
valid_loss_min = np.Inf
train_losses = []
valid_losses = []
avg_train_losses = []
avg_valid_losses = []
train_acc, valid_acc = [], []
best_acc = 0.0
early_stopping = EarlyStopping(patience=5, verbose=True)

In [9]:
for epoch in range(epochs):
    model.train()
    total_train = 0
    correct_train = 0
    total_valid = 0
    correct_valid = 0

    for step, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        predicted = outputs.argmax(1)
        total_train += labels.nelement()
        correct_train += (predicted == labels).sum().item()
        train_accuracy = correct_train / total_train
        
    model.eval()
    with torch.no_grad():
        accuracy = 0
        for inputs, labels in valid_loader:
            
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_losses.append(loss.item())
            # Calculate accuracy
            predicted = outputs.argmax(1)
            total_valid += labels.nelement()
            correct_valid += (predicted == labels).sum().item()
            valid_accuracy = correct_valid / total_valid
        
    train_loss = np.average(train_losses)
    valid_loss = np.average(valid_losses)
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)
    valid_acc.append(valid_accuracy) 
    train_acc.append(train_accuracy)

    # calculate average losses
    
    # print training/validation statistics 
    print(f"Epoch {epoch+1}/{epochs}.. ")
    print('Training Loss: {:.6f} \tValidation Loss: {:.6f} \tTrraining Accuracy: {:.6f} \tValidation Accuracy: {:.6f}'.format(
        train_loss, valid_loss, train_accuracy*100, valid_accuracy*100))
    train_losses = []
    valid_losses = []        
    if valid_accuracy > best_acc:
        best_acc = valid_accuracy
    early_stopping(valid_loss, args, model, optimizer, save=False)
    if early_stopping.early_stop:
        print("Early stopping")
        break
    
print('Best val Acc: {:4f}'.format(best_acc*100))  

Epoch 1/10000.. 
Training Loss: 0.933960 	Validation Loss: 1.078514 	Trraining Accuracy: 56.875000 	Validation Accuracy: 48.592593
Epoch 2/10000.. 
Training Loss: 0.563557 	Validation Loss: 1.221853 	Trraining Accuracy: 77.867647 	Validation Accuracy: 36.148148
EarlyStopping counter: 1 out of 5
Epoch 3/10000.. 
Training Loss: 0.328918 	Validation Loss: 1.333228 	Trraining Accuracy: 88.051471 	Validation Accuracy: 39.703704
EarlyStopping counter: 2 out of 5
Epoch 4/10000.. 
Training Loss: 0.218179 	Validation Loss: 1.486598 	Trraining Accuracy: 92.058824 	Validation Accuracy: 45.037037
EarlyStopping counter: 3 out of 5
Epoch 5/10000.. 
Training Loss: 0.139501 	Validation Loss: 1.063093 	Trraining Accuracy: 95.330882 	Validation Accuracy: 59.703704
Epoch 6/10000.. 
Training Loss: 0.097880 	Validation Loss: 0.775117 	Trraining Accuracy: 96.838235 	Validation Accuracy: 72.888889
Epoch 7/10000.. 
Training Loss: 0.080967 	Validation Loss: 0.799303 	Trraining Accuracy: 97.316176 	Validation A

In [10]:
def print_specificity(specificity):
    print('\t\tspecificity')
    print('')

    print(f'       covid\t{specificity[0]:.2f}')
    print(f'     healthy\t{specificity[1]:.2f}')
    print(f'      others\t{specificity[2]:.2f}')
    print('')

    macro_specificity = sum(specificity) / 3.0
    print(f'   macro avg\t{macro_specificity:.2f}')

    weighted = [434/835, 152/835, 249/835] 
    weighted_specificity = weighted @ specificity
    print(f'weighted avg\t{weighted_specificity:.2f}')
    print('')

In [11]:
loss_epoch = 0
accuracy_epoch = 0
model.eval()
pred = []
true = []
soft = []
for step, (x, y) in enumerate(test_loader):
    model.zero_grad()

    x = x.to(args.device)
    y = y.to(args.device)

    outputs = model(x)
    loss = criterion(outputs, y)
    
    # for majority voting
    softmax = torch.nn.Softmax(dim=1)
    s = softmax(outputs).cpu().detach().tolist()
    for i in range(len(s)):
        soft.append(s[i])

    predicted = outputs.argmax(1)
    preds = predicted.cpu().numpy()
    labels = y.cpu().numpy()
    preds = np.reshape(preds, (len(preds), 1))
    labels = np.reshape(labels, (len(preds), 1))

    for i in range(len(preds)):
        pred.append(preds[i][0].item())
        true.append(labels[i][0].item())
    
    acc = (predicted == y).sum().item() / y.size(0)
    accuracy_epoch += acc

    loss_epoch += loss.item()

cnf_matrix = confusion_matrix(true, pred)
print('Confusion Matrix:\n', cnf_matrix)

FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

accuracy_epoch = np.diag(cnf_matrix).sum().item() / len(true)

# Specificity or true negative rate
specificity = TN/(TN+FP) 

print_specificity(specificity)

report = classification_report(true, pred, target_names=['covid', 'healthy', 'others'])
print(report)

Confusion Matrix:
 [[324  22  56]
 [  2 118  13]
 [ 43  35 163]]
		specificity

       covid	0.88
     healthy	0.91
      others	0.87

   macro avg	0.89
weighted avg	0.88

              precision    recall  f1-score   support

       covid       0.88      0.81      0.84       402
     healthy       0.67      0.89      0.77       133
      others       0.70      0.68      0.69       241

    accuracy                           0.78       776
   macro avg       0.75      0.79      0.77       776
weighted avg       0.79      0.78      0.78       776

